## Get Credentials (Effectivly Drafts)
In this code, we create a way to get our credentials to add to our scraper.

Output: We succesfully have needed credentials in a dict format!

### First Way (Get Next Acccount from most recent last_used timestamp)

In [3]:
import sqlite3
from cryptography.fernet import Fernet
import os
import datetime


# Open the key file to read the encryption key
with open("../credentials/key.key", "rb") as key_file:
    key = key_file.read()

# Initialize the Fernet cipher with the key
fernet = Fernet(key)

# Get next account automatically from the db by selecting the least recently used account that is still active
def get_next_account():
    conn = sqlite3.connect("../database/scraper.db")
    cursor = conn.cursor()

    # Select the least recently used account that’s still active
    cursor.execute("""
        SELECT id, username, encrypted_password, proxy
        FROM accounts
        WHERE status = 'active'
        ORDER BY last_used ASC NULLS FIRST
        LIMIT 1
    """)
    # fetch the first row from the results
    row = cursor.fetchone()

    # if the row is None, it means there are no active accounts
    if row is None:
        conn.close()
        raise Exception("No active accounts available.")
    
    # Unpack the row into variables
    account_id, username, encrypted_password, proxy = row

    # Decrypt the password
    password = fernet.decrypt(encrypted_password.encode()).decode()

    # Update the row with the current timestamp
    cursor.execute("""
        UPDATE accounts
        SET last_used = ?
        WHERE id = ?
    """, (datetime.datetime.now().isoformat(), account_id))
    conn.commit()
    conn.close()

    # return the account details as a dictionary
    return {
        "id": account_id,
        "username": username,
        "password": password,
        "proxy": proxy
    }


### Second Way (Get Account data by id)
This is the one that I am going to use and the one I converted to a .py

In [ ]:
def get_account(id):
    conn = sqlite3.connect("../database/scraper.db")
    cursor = conn.cursor()

    # Select the account by id that is still active
    cursor.execute("""
        SELECT id, username, encrypted_password, proxy
        FROM accounts
        WHERE id = ? AND status = 'active'
        ORDER BY last_used ASC NULLS FIRST
        LIMIT 1
    """, (id,))

    # get the first row from all the results (there should only be one row)
    row = cursor.fetchone()

    # if no row is returned, raise an exception
    if row is None:
        conn.close()
        raise Exception("No account with that id.")

    # unpack the row (which is a tuple) into variables
    account_id, username, encrypted_password, proxy = row

    # Decrypt the password
    password = fernet.decrypt(encrypted_password.encode()).decode()

    # Update last_used
    cursor.execute("""
        UPDATE accounts
        SET last_used = ?
        WHERE id = ?
    """, (datetime.datetime.now().isoformat(), account_id))
    conn.commit()
    conn.close()

    # return the account details as a dictionary
    return {
        "id": account_id,
        "username": username,
        "password": password,
        "proxy": proxy
    }

In [ ]:
get_account(1)  # Example usage, for testing purposes, replace with the actual account ID you want to retrieve

{'id': 1,
 'username': 'ElizabethMoore2247',
 'password': "'=l/yt{tq<'O",
 'proxy': '156.228.114.248:3129'}